This will be the first study for the dataset which we have just extracted.
In this work, we begin with a Feature Extractor model. Also, we will be using Clustering Algorihtm with
~ Melih Kaçaman - 12/12/2022

# Creating Data Loaders

# Setting Up Callbacks (things to run whilst our model trains)

- Model checkpoint with the ModelCheckPoint callback


In [2]:
from re import T
# Create Tensorboard callback
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving Tensorboard log files to: {log_dir}")
    return tensorboard_callback

# Creating Models Using Keras Functional API

In [5]:
SHAPE_WITH_CHANNELS = (224, 224, 3)
SHAPE = (224, 224)

In [6]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
# I'm not going to fine-tune this model.
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=SHAPE_WITH_CHANNELS, name='input_layer')
x = base_model(inputs)
outputs = tf.keras.layers.GlobalAvgPool2D(name='global_average_pooling_layer')(x)

model_0 = tf.keras.Model(inputs, outputs)

16705208/16705208 [==============================] - 16s 1us/step


In [10]:
img_to_array_custom = tf.keras.preprocessing.image.img_to_array
load_img_custom = tf.keras.preprocessing.image.load_img

In [12]:
import tensorflow_hub as hub
import numpy as np

def get_embeddings_img(path):
    model = hub.KerasLayer('https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2')
    image = load_img_custom(path, target_size=SHAPE)
    image = img_to_array_custom(image)
    image = np.expand_dims(image, axis=0)

    embeddings = model(image)

    return embeddings

In [14]:
path = '../datasets/dataset_523/brand_08/1-0TK-VR033/0.jpg'
res = get_embeddings_img(path)
res.shape

<tf.Tensor: shape=(1, 1280), dtype=float32, numpy=
array([[0.03808071, 0.        , 0.52875704, ..., 0.        , 1.4017409 ,
        0.        ]], dtype=float32)>

In [18]:
type(res)

tensorflow.python.framework.ops.EagerTensor

In [21]:
type(res.numpy())

numpy.ndarray

In [22]:
res.numpy()

array([[0.03808071, 0.        , 0.52875704, ..., 0.        , 1.4017409 ,
        0.        ]], dtype=float32)

In [37]:
base_model.layers

In [23]:
path = '../datasets/dataset_cv3/brand_08/1-0CD-VR029/2145.png'
res2 = get_embeddings_img(path)
res2.shape

TensorShape([1, 1280])

In [25]:
dir = '../datasets/dataset_cv3/brand_08/'
data_tf = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dir,
    image_size=(224, 224),
    label_mode='categorical',
    batch_size=32
)

Found 1108 files belonging to 109 classes.


In [28]:
type(data_tf)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [32]:
embeddings = model_0.predict(data_tf)

35/35 [==============================] - 64s 2s/step


- Embeddings shapes are correct, so we can move on with tensorflow batch-dataset.

In [34]:
data_tf.class_names

['1-011-200',
 '1-011-600',
 '1-011-VR013',
 '1-011-VR028',
 '1-011-VR030',
 '1-011-VR033',
 '1-011-VR090',
 '1-013-VR033',
 '1-013-VR046',
 '1-013-VR086',
 '1-016-200',
 '1-016-VR014',
 '1-016-VR027',
 '1-016-VR029',
 '1-016-VR033',
 '1-016-VR046',
 '1-016-VR099',
 '1-033-001',
 '1-033-200',
 '1-033-400',
 '1-033-600',
 '1-033-VR013',
 '1-033-VR024',
 '1-033-VR027',
 '1-033-VR029',
 '1-033-VR033',
 '1-033-VR046',
 '1-035-001',
 '1-073-VR033',
 '1-073-VR046',
 '1-080-DN0022',
 '1-080-DN0023',
 '1-082-VR033',
 '1-0CD-VR014',
 '1-0CD-VR029',
 '1-0CD-VR033',
 '1-0CD-VR046',
 '1-0CD-VR099',
 '1-0GU-200',
 '1-0GU-300',
 '1-0GU-VR003',
 '1-0GU-VR013',
 '1-0GU-VR014',
 '1-0GU-VR027',
 '1-0GU-VR030',
 '1-0GU-VR033',
 '1-0GU-VR036',
 '1-0GU-VR054',
 '1-0HF-VR033',
 '1-0MS-VR033',
 '1-0OP-VR033',
 '1-0TK-VR033',
 '1-0TK-VR086',
 '1-B11-VR013',
 '1-B11-VR033',
 '1-B11-VR046',
 '1-BGU-VR003',
 '1-BGU-VR013',
 '1-BGU-VR033',
 '2-011-600',
 '2-011-VR013',
 '2-011-VR033',
 '2-013-VR033',
 '2-016-VR03

In [42]:
embeddings.shape

(1108, 1280)

In [60]:
type(embeddings)

numpy.ndarray

# Cluestering 

## Model C_1: 
- We will be dealing with the dataset that includes just extracted inputs using K-Means. 

In [62]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=len(data_tf.class_names)).fit(embeddings)

kmeans_model.labels_

array([78, 52, 22, ..., 83, 83, 21])

In [63]:
kmeans_model.get_feature_names_out() 

AttributeError: 'KMeans' object has no attribute 'get_feature_names_out'